#1-Chatbot_InterviewPreparation

In [1]:
!pip install openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.6 MB/s eta 0:00:00


In [2]:
!pip install pydantic==1.10
!pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 38.7 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [3]:
import os
import json
import pandas as pd
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
import openai
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
from typing import List, Union, Any, Optional, Type
import re
from langchain import PromptTemplate
from langchain.tools import BaseTool
from langchain.chains import LLMChain

from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

In [4]:
os.environ["OPENAI_API_KEY"] = "Your API Key"

In [25]:
class BaseAgent():
    def __init__(self, cv, job_offer,categories,n_questions,client):
        self.cv = cv
        self.job_offer = job_offer
        self.categories=categories
        self.n_questions=n_questions
        self.client = client


class QuestionGeneratorAgent(BaseAgent):
    def __init__(self, cv, job_offer, categories, n_questions,client):
        super().__init__(cv, job_offer, categories, n_questions,client)

        self.template_question_generation = """you are an non-technical interviewer  conducting a job interview.
         You will be provided with the cv {cv} and the job offer {job_offer}
         and will generate a number of questions {n_questions} for each of the following categories {categories}, only the number specified as number of questions.
         You shoul take into consideration the cv and the job offer to generate valid and valuable questions across all the categories.
         You answer strictly as single JSON string. Don't include any other verbose texts and don't include the markdown syntax anywhere.JSON format: ["question":, "category":].
        before generating the question make sure only include the specified numer of questions {n_questions} per category.
        """
        self.prompt_question_generation = PromptTemplate(template=self.template_question_generation, input_variables=["cv", "job_offer","categories","n_questions"])

        self.llm_chain_question_generation = LLMChain(prompt=self.prompt_question_generation, llm=self.client)

    def _generate_questions(self) -> list[dict[str, str]]:


        try:
            generated_questions_json = self.llm_chain_question_generation.run({'cv': self.cv, "job_offer": self.job_offer,"categories":self.categories,"n_questions":self.n_questions})
            questions_parsed = json.loads(generated_questions_json)
            return questions_parsed


        except Exception as e:
            print(e)
            return "Error generating questions"




class DialogueManager():
    def __init__(self, questions):
        self.questions = questions

    def _run_interview(self):
        interview_records = []
        for question in self.questions:
            user_reply = input(f"{question['question']} ")
            interview_records.append({
                "question": question['question'],
                "category": question['category'],
                "user_reply": user_reply
            })


        return interview_records



class Evaluation(BaseAgent):
    def __init__(self,job_offer, categories, client,interview_records):
          super().__init__(None,None,job_offer, categories, client)
          self.interview_records=interview_records


          self.template_evaluation_generation = """you are an evaluator of job interview, you will be provided with the job offer {job_offer}
             and the responses given by the user in the interview records {interview_records} that include ('question,'category','user_reply').
             Your task is based on the information provided :
             Evaluate critically the answers for every category in category {categories} and provide output in the following JSON format based on the adequacy of the responses to the questions and the job offer:
            (
                "Evaluation": "general evaluation of the answer, exclusively based on the user replies",
                "Reason": "reasons behind the evaluation",
                "Feedback": "Feedback or suggestions for improvement",
                
            )
            your evaluation should be exclusively based on the replies provided in the interview records and should not include external additions.
            """



          self.prompt_evaluation_generation = PromptTemplate(template=self.template_evaluation_generation, input_variables=["job_offer", "categories","interview_records"])

          self.llm_chain_evaluation_generation = LLMChain(prompt=self.prompt_evaluation_generation, llm=self.client)

    def _generate_evaluation(self) -> list[dict[str, str]]:
            try:
                evaluation = self.llm_chain_evaluation_generation.run({"job_offer": self.job_offer, "categories":self.categories, "interview_records": self.interview_records})
                parsed_evaluation = json.loads(evaluation)
                return parsed_evaluation

            except Exception as e:
                print(e)
                return " Error generating Evaluation"

In [26]:
def interview_pipeline(cv_info, job_offer_info, categories_list,n_questions,ai_client):


# Use the ai_client from BaseAgent to activate the question generator
    question_generator = QuestionGeneratorAgent(cv_info, job_offer_info, categories_list,n_questions, ai_client)

# Generate questions
    generated_questions = question_generator._generate_questions()


    interviewer = DialogueManager(generated_questions)
    interview_records = interviewer._run_interview()
    print(interview_records)

    evaluator = Evaluation(job_offer_info, categories_list, ai_client,interview_records)
    evaluation_result = evaluator._generate_evaluation()

    print("Generated Questions:")
    print(generated_questions)
    print("Evaluation Result:")
    print(evaluation_result)
    return evaluation_result



In [27]:
cv="With extensive expertise spanning talent acquisition, employee relations, and HR operations, I am a seasoned HR Specialist with a track record of success. Armed with a rich background of [X years] in the field, I specialize in crafting and implementing robust HR strategies aimed at attracting top-tier talent, bolstering retention rates, and nurturing a thriving work culture. My forte lies in streamlining HR processes, optimizing efficiency, and fostering career growth among employees. Noteworthy achievements include orchestrating recruitment initiatives that elevated candidate quality by 30%, alongside spearheading engagement programs that yielded a remarkable 20% improvement in retention rates. My educational background includes an MBA and a BSc in Human Resources Management, complemented by certifications such as the SHRM Certified Professional (SHRM-CP) and other relevant credentials. My skill set encompasses talent acquisition, performance management, HRIS administration, policy development, and effective communication, making me a valuable asset adept at navigating the dynamic landscape of human resources."
job_offer="Seeking a vibrant addition to our team, is on the lookout for a seasoned Human Resources Specialist. This pivotal role involves driving talent acquisition, nurturing employee relations, and optimizing HR operations to bolster our company's growth trajectory. We prioritize fostering a positive workplace culture and innovation across all facets of our operations. The ideal candidate will spearhead recruitment strategies, implement engaging employee programs, manage performance appraisals, and streamline HR processes. A Bachelor's degree in Human Resources Management or a related field, coupled with [X years] of HR experience, is required, with expertise in HRIS systems, talent sourcing tools, and strong communication skills. Certifications like SHRM-CP or PHR would be advantageous. We offer competitive compensation, comprehensive benefits, and a supportive work environment that champions professional development. Join us in shaping an environment where talent thrives and success blossoms"
categories=["Personal","Role specific","Behavioural","Situational"]
n_questions=1
ai_client = ChatOpenAI(model_name="gpt-3.5-turbo")

outcome=interview_pipeline(cv, job_offer, categories,1,ai_client)
outcome

Can you tell me about your experience in talent acquisition and employee relations? I worked as an Head-hunter for over 5 years
What strategies have you implemented to attract top-tier talent and improve retention rates? I encourage team-work and active participations in the decision making of the whole company
Can you provide an example of a recruitment initiative you orchestrated that resulted in an improvement in candidate quality? I introduced a multi-department transper program
How would you handle a situation where an employee is not meeting performance expectations? I would talk to the person to understand the reasons
[{'question': 'Can you tell me about your experience in talent acquisition and employee relations?', 'category': 'Personal', 'user_reply': 'I worked as an Head-hunter for over 5 years'}, {'question': 'What strategies have you implemented to attract top-tier talent and improve retention rates?', 'category': 'Role specific', 'user_reply': 'I encourage team-work and a

{'Evaluation': [{'Category': 'Personal',
   'Evaluation': 'Adequate',
   'Reason': 'The user has relevant experience as a head-hunter for over 5 years, which aligns with the requirement for talent acquisition.',
   'Feedback': 'None'},
  {'Category': 'Role specific',
   'Evaluation': 'Inadequate',
   'Reason': "The user's response does not directly answer the question about strategies implemented to attract top-tier talent and improve retention rates. Encouraging teamwork and active participation is not specific enough.",
   'Feedback': 'The candidate should provide specific examples of strategies implemented to attract top-tier talent and improve retention rates.'},
  {'Category': 'Behavioural',
   'Evaluation': 'Adequate',
   'Reason': 'The user provided an example of introducing a multi-department transparency program, which showcases a recruitment initiative that resulted in an improvement in candidate quality.',
   'Feedback': 'None'},
  {'Category': 'Situational',
   'Evaluation'